<a href="https://colab.research.google.com/github/naveedkhalid091/Learn_Agentic_AI/blob/main/step02_generative_ai_for_beginners%20/02(c)_Advance_RAG(responding_from_Database_content_only).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Advance Rag:**

Here we will learn to create the RAG pipeline where LLM will only respond from the available dataset into the databases, means it will respond on whatever we have given the inforamtion.


In [1]:
!pip install -U -q google-generativeai

In [2]:

from google.colab import userdata
import os
os.environ['GOOGLE_API_KEY']=userdata.get('GOOGLE_API_KEY')

## **Selection of the right model for advance RAG functionalities:**

For the right model selection you can print the available models using below code:

The model's usage and limitations will also be mentioned.

- When you will navigate to `embedding model 001`, it will be mentioned that it returns the `embed_contents` & its input and output token limitiations will also be mentioned etc.
- When you will navigate to `embedding model 004`, You can read all the features of this model as well.




In [ ]:
import google.generativeai as genai

list(genai.list_models())


In [ ]:
# Simple Embeddings without langchain
from typing import Dict

result:Dict=genai.embed_content(
    model="models/text-embedding-004",  # compulsory
    content="What is the meanning of life",                       # compulsory
    task_type="retrieval_document",   # compulsory
    title="Embedding of single string",  # optional_parameter
)

result

In [ ]:
len(result['embedding'])  # it will print the lenght of embedding content, every content will consist of 768 digit's length

**Note:** If more then one content is required then what will be written in an array like `content=["What is the meanning of life", "I love Pakistan"]` and when you will check the lenght the it will return the number of content not the digit's lenght.

Lets see via coding example:

In [ ]:
# Simple embeddings witohut langchain

from typing import Dict

result_2:Dict=genai.embed_content(
    model="models/text-embedding-004",  # compulsory
    content=["What is the meanning of life",
             "I love Pakistan",
             "Pakistan Zindabaad"
             ],  # compulsory
    task_type="retrieval_document",   # compulsory
    title="Embedding of single string",  # optional_parameter
)

result_2

In [ ]:
# checking the lenght
len(result_2['embedding'])

In [ ]:
# Checking the lenght of each content in a list

for v in result_2['embedding']:
  print(len(v))

## **Building vector stores, Retrieval using Chroma DB and Langchain:**

In [ ]:
# working with chroma database (langchain supported)

!pip install -U -q langchain-chroma

In [10]:
## for small text you can import Document from the langchain.

from langchain_core.documents import Document

document_1=Document(
    page_content="Dogs are great companions, known for their loyalty",
    metadata={"source":"mammal-pets-doc"}
)

document_2=Document(
    page_content="Cats are independent pets that often enjoy their own space",
    metadata={"source":"mammal-pets-doc"}
)

documents=[document_1, document_2]



In [11]:
!pip install -U -q langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.6 MB/s eta 0:00:00


In [12]:
# emdedding through langchain
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings=GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

In [ ]:
embeddings.embed_query("What is the meaning of life?")

In [14]:
from langchain_chroma import Chroma


vector_store=Chroma.from_documents(documents=documents, embedding=embeddings)


In [ ]:
# Checking functions of vector store: these functions will be created automatically.

list(dir(vector_store))

In [ ]:
vector_store.similarity_search("tell me about cat")

In [ ]:
# Checking through async function

await vector_store.asimilarity_search("tell me about cat")

In [ ]:
vector_store.similarity_search_with_score("tell me about cat")

In [19]:
embedding=embeddings.embed_query("cat")  # convert cat into vector

In [ ]:
vector_store.similarity_search_by_vector(embedding)

In [ ]:
embedding

## **Retriever:**

In [ ]:
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever =RunnableLambda(vector_store.similarity_search).bind(k=1)

retriever.batch("cats")


In [23]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash",
                           api_key=userdata.get("GOOGLE_API_KEY")
                           )

## **Augment part:**

In [24]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# llm will now respond based on the provided document because of the following message.

message="""
Answer this question using the provided context only.
{question}

context:
{context}
 """

In [25]:
prompt=ChatPromptTemplate.from_messages([("human",message)])

# Building RAG:

In [26]:
              # Retriever part                                    # Augment  # LLM generation
rag_chain={"context":retriever, "question":RunnablePassthrough()} | prompt | llm

In [27]:
response=rag_chain.invoke("tell me about Allama Iqbal")

print(response.content)

This document does not contain any information about Allama Iqbal.


In [28]:
response=rag_chain.invoke("tell me about cats")

print(response.content)

Based on the provided text, cats are independent pets that often enjoy their own space.


In [29]:
response=rag_chain.invoke("tell me about Dogs")

print(response.content)

Based on the provided text, dogs are great companions known for their loyalty.


**Note:** We can also invoke from above llm object but that llm will response based on its training data. It will not response based on your Documents/RAG system.

**For Example:**



In [30]:
response=llm.invoke("tell me about Allama Iqbal in one line")

print (response.content)    # Note that llm is not responding based on our data RAG system,

Allama Iqbal was a highly influential poet, philosopher, and political activist whose work profoundly shaped the ideology of Pakistan.
